In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Dropdown, fixed
from IPython.display import display
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go

In [2]:
# Path to go up one level from the current folder to the parent folder
parent_path = os.path.join(os.getcwd(), '..')

# Path to the target data folder
data_folder_path = os.path.join(parent_path, 'Data')

In [3]:
menu_df = pd.read_csv(os.path.join(data_folder_path, 'menu_analysis.csv'))
customers_df = pd.read_csv(os.path.join(data_folder_path, 'customers.csv'))
stores_df = pd.read_csv(os.path.join(data_folder_path, 'stores.csv'))
structure_df = pd.read_csv(os.path.join(data_folder_path, 'structure.csv'))
size_df = pd.read_csv(os.path.join(data_folder_path, 'order_size.csv'))
transactions_df = pd.read_csv(
        os.path.join(data_folder_path, "transactions.csv"),
        parse_dates=['Transaction_Date']  # Modify as needed if format issues arise
    )

### Modification 1

In [4]:
transactions_df['Transaction_Date'] = pd.to_datetime(transactions_df['Transaction_Date'])

### Modification 2

In [5]:
working_df = pd.merge(transactions_df, menu_df[['Item_Number', 'Price']], )

### Modification 3

In [7]:
daily_sales = working_df.groupby(['Transaction_Date', 'Store_Number'])['Price'].sum().reset_index()

,Transaction_Date,Store_Number,Price
0,2023-06-21,STR_1,331.5
1,2023-06-21,STR_10,344.0
2,2023-06-21,STR_2,507.0
3,2023-06-21,STR_3,306.0
4,2023-06-21,STR_4,403.5


In [19]:
round(daily_sales['Price'].max() * 1.25 , -2)

1000.0

# 2 Daily Sales Distribution

In [20]:
fig = go.Figure()

for store in daily_sales['Store_Number'].unique():
    store_data = daily_sales[daily_sales['Store_Number'] == store]
    fig.add_trace(go.Box(
        y=store_data['Price'],
        name=f'Store {store}',
        boxpoints='all',
        jitter=0.3,
        pointpos=-1.8
    ))

fig.update_layout(
    title='Daily Sales Distribution by Store',
    yaxis_title='Daily Sales ($)',
    xaxis_title='Store',
    showlegend=False,
    hovermode='closest',
    yaxis=dict(
        range=[0, round(daily_sales['Price'].max() * 1.25 , -2)],  # Set your range based on the data
        title='Daily Sales ($)'
    )
)

fig.show()